In [1]:
import numpy as np
import random

def encode_message(message, generator_poly):
    return np.polymul(message, generator_poly) % 2


def decode_message(encoded_msg, generator_poly, max_error_count, is_block_error):
    syndrome = np.polydiv(encoded_msg, generator_poly)[1] % 2
    return correct_errors_in_message(encoded_msg, syndrome, generator_poly, max_error_count, is_block_error)


def flip_bits(bits, positions):
    for pos in positions:
        bits[pos] ^= 1  
    return bits


def introduce_random_errors(bits, error_count):
    positions = random.sample(range(len(bits)), error_count)
    print(f"Ошибки добавлены в позиции: {positions}")
    return flip_bits(bits, positions)


def introduce_error_block(bits, block_length):
    size = len(bits)
    start = random.randint(0, size - block_length)
    end = (start + block_length - 1) % size
    print(f"Блок ошибок добавлен в диапазоне: {start}-{end}")
    for offset in range(block_length):
        bits[(start + offset) % size] ^= 1  
    return bits


def remove_zeros(arr):
    return np.trim_zeros(np.trim_zeros(arr, 'f'), 'b')


def can_detect_error(syndrome, max_error_count):
    trimmed_syndrome = remove_zeros(syndrome)
    return 0 < len(trimmed_syndrome) <= max_error_count


def correct_errors_in_message(encoded_msg, syndrome, generator_poly, max_error_count, is_block_error):
    msg_length = len(encoded_msg)

    for shift in range(msg_length):
        error_poly = np.zeros(msg_length, dtype=int)
        error_poly[msg_length - shift - 1] = 1
        shifted_syndrome = np.polymul(syndrome, error_poly) % 2
        reduced_syndrome = np.polydiv(shifted_syndrome, generator_poly)[1] % 2

        if is_block_error and can_detect_error(reduced_syndrome, max_error_count):
            return apply_correction(encoded_msg, reduced_syndrome, shift, generator_poly)
        elif not is_block_error and sum(reduced_syndrome) <= max_error_count:
            return apply_correction(encoded_msg, reduced_syndrome, shift, generator_poly)

    return None  


def apply_correction(encoded_msg, reduced_syndrome, shift, generator_poly):
    msg_length = len(encoded_msg)
    correction_poly = np.zeros(msg_length, dtype=int)
    correction_poly[shift - 1] = 1
    correction = np.polymul(correction_poly, reduced_syndrome) % 2
    corrected_msg = np.polyadd(correction, encoded_msg) % 2
    return np.array(np.polydiv(corrected_msg, generator_poly)[0] % 2).astype(int)


def execute_error_correction_analysis(generator_poly, message, error_func, error_param, max_error_count,
                                      is_block_error):
    print(f"Исходное сообщение: {message}")
    encoded_msg = encode_message(message, generator_poly)
    print(f"Закодированное сообщение: {encoded_msg}")

    erroneous_msg = error_func(encoded_msg.copy(), error_param)
    print(f"Сообщение с ошибками: {erroneous_msg}")

    decoded_msg = decode_message(erroneous_msg, generator_poly, max_error_count, is_block_error)
    print(f"Декодированное сообщение: {decoded_msg}")

    if np.array_equal(message, decoded_msg):
        print("Декодирование успешно. Сообщение совпадает.\n")
    else:
        print("Ошибка в декодировании. Сообщение не совпадает.\n")


def test_7_4_code():
    print("Тестирование кода (7,4)")
    generator_poly = np.array([1, 1, 0, 1])
    message = np.array([1, 0, 1, 0])

    for error_count in range(1, 4): 
        execute_error_correction_analysis(generator_poly, message, introduce_random_errors, error_count, 1, False)


def test_15_9_code():
    print("Тестирование кода (15,9)")
    generator_poly = np.array([1, 0, 0, 1, 1, 1, 1])
    message = np.array([1, 1, 0, 0, 0, 1, 0, 0, 0])

    for block_length in range(1, 5): 
        execute_error_correction_analysis(generator_poly, message, introduce_error_block, block_length, 3, True)

In [2]:
test_7_4_code()

Тестирование кода (7,4)
Исходное сообщение: [1 0 1 0]
Закодированное сообщение: [1 1 1 0 0 1 0]
Ошибки добавлены в позиции: [2]
Сообщение с ошибками: [1 1 0 0 0 1 0]
Декодированное сообщение: [1 0 1 0]
Декодирование успешно. Сообщение совпадает.

Исходное сообщение: [1 0 1 0]
Закодированное сообщение: [1 1 1 0 0 1 0]
Ошибки добавлены в позиции: [4, 6]
Сообщение с ошибками: [1 1 1 0 1 1 1]
Декодированное сообщение: [1 0 1 1]
Ошибка в декодировании. Сообщение не совпадает.

Исходное сообщение: [1 0 1 0]
Закодированное сообщение: [1 1 1 0 0 1 0]
Ошибки добавлены в позиции: [5, 3, 4]
Сообщение с ошибками: [1 1 1 1 1 0 0]
Декодированное сообщение: [1 1 0 0]
Ошибка в декодировании. Сообщение не совпадает.



In [3]:
test_15_9_code()

Тестирование кода (15,9)
Исходное сообщение: [1 1 0 0 0 1 0 0 0]
Закодированное сообщение: [1 1 0 1 0 1 0 1 1 1 1 1 0 0 0]
Блок ошибок добавлен в диапазоне: 4-4
Сообщение с ошибками: [1 1 0 1 1 1 0 1 1 1 1 1 0 0 0]
Декодированное сообщение: [1 1 0 0 0 1 0 0 0]
Декодирование успешно. Сообщение совпадает.

Исходное сообщение: [1 1 0 0 0 1 0 0 0]
Закодированное сообщение: [1 1 0 1 0 1 0 1 1 1 1 1 0 0 0]
Блок ошибок добавлен в диапазоне: 2-3
Сообщение с ошибками: [1 1 1 0 0 1 0 1 1 1 1 1 0 0 0]
Декодированное сообщение: [1 1 0 0 0 1 0 0 0]
Декодирование успешно. Сообщение совпадает.

Исходное сообщение: [1 1 0 0 0 1 0 0 0]
Закодированное сообщение: [1 1 0 1 0 1 0 1 1 1 1 1 0 0 0]
Блок ошибок добавлен в диапазоне: 7-9
Сообщение с ошибками: [1 1 0 1 0 1 0 0 0 0 1 1 0 0 0]
Декодированное сообщение: [1 1 0 0 0 1 0 0 0]
Декодирование успешно. Сообщение совпадает.

Исходное сообщение: [1 1 0 0 0 1 0 0 0]
Закодированное сообщение: [1 1 0 1 0 1 0 1 1 1 1 1 0 0 0]
Блок ошибок добавлен в диапазоне: 